In [1]:
import pandas as pd
import psycopg2
import plotly.express as px

По результатам исследования базы данных с вакансиями можно сделать следующие выводы:

Большинство вакансий встречается в крупных городах, где находится большое количество различных компаний.
ТОП-5 популярных вакансий: Системный администратор, Программист 1С, Бизнес-аналитик, Специалист технической поддержки, Системный аналитик. Это свидетельствует о востребованности IT специалистов на рынке труда.
Чаще других вакансии размещают работодатели, у которых указаны сферы деятельности: "Разработка программного обеспечения" "Системная интеграция. IT-консалтинг", "Интернет-компания".
Основная масса всех вакансий это: регион - крупный город, тип рабочего графика и трудоустройства - полная занятость/полный рабочий день, опыт работы - от 1 года до 3 лет.
Примерно в половине случаев работодатель не указывает уровень заработной платы. Вероятнее всего, это будет определяться по результатам собеседования.
Для начинающего специалиста по Data Science есть возможность выбора из 51 вакансии.
Начинающий дата-саентист, желающий найти удаленную работу, должен иметь опыт не менее 1 года. Он может рассчитывать на зарплату от 75 000 рублей.
В качестве продолжения исследования было бы интересно выяснить средний уровень зарплаты в разрезе регионов и сфер деятельнсти/компаний.

Дополнительно узнаем минимальный и максимальный уровень зарплаты для дата-саентиста с типом рабочего графика Удаленная работа

In [ ]:
query_7 = f'''
            SELECT experience,
            ROUND(MIN(COALESCE((salary_from + salary_to)/2, salary_to, salary_from))) min_salary,
            ROUND(MAX(COALESCE((salary_from + salary_to)/2, salary_to, salary_from))) max_salary                    
            FROM public.vacancies
            WHERE (name ILIKE '%data scientist%'
                OR name ILIKE '%data science%'
                OR name ILIKE '%исследователь данных%'
                OR (name LIKE '%ML%' AND name NOT LIKE'%HTML%')
                OR name ILIKE '%machine learning%'
                OR name ILIKE '%машинн%обучен%')
                AND (salary_to IS NOT NULL OR salary_from IS NOT NULL)
                AND schedule = 'Удаленная работа'
            GROUP BY experience
            '''

In [ ]:
# результат запроса
ds_salary_remote = pd.read_sql_query(query_7, connection)
ds_salary_remote

Дополнительно определим какие сферы деятельности чаще всего работодатели указывают в своих вакансиях

In [ ]:
query_8 = f'''
            SELECT i.name industry_name,
                COUNT(v.id) cnt_vacancies
            FROM public.industries i
            JOIN public.employers_industries ei ON i.id = ei.industry_id
            JOIN public.employers e ON e.id = ei.employer_id
            JOIN public.vacancies v ON e.id = v.employer_id
            GROUP BY i.name
            ORDER BY cnt_vacancies DESC
            '''

In [ ]:
# результат запроса
industries = pd.read_sql_query(query_8, connection)
industries

In [ ]:
# построим график
fig = px.treemap(
    data_frame=industries, 
    path=['industry_name'], 
    values='cnt_vacancies', 
    height=800, 
    width=1200, 
    title='Сферы деятельности работодателей, указанные в вакансиях' #заголовок
)

#отображаем график
fig.show()

Определим ТОП-5 популярных вакансий

In [ ]:
query_9 = f'''
            SELECT name vacancies_name,
                COUNT(id) cnt_vacancies
            FROM public.vacancies
            GROUP BY name
            ORDER BY cnt_vacancies DESC
            LIMIT 5
            '''

In [ ]:
# результат запроса
top_vacancy = pd.read_sql_query(query_9, connection)
top_vacancy

In [ ]:
# connection.close()